In [1]:
# pip install -q google-adk[a2a]


In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---

# get_product_info

In [5]:
# Define a product catalog lookup tool
# In a real system, this would query the vendor's product database
def get_product_info(product_name: str) -> str:
    """Get product information for a given product.

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Product information as a string
    """
    # Mock product catalog - in production, this would query a real database
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": 'Dell XPS 15, $1,299, In Stock (45 units), 15.6" display, 16GB RAM, 512GB SSD',
        "macbook pro 14": 'MacBook Pro 14", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD',
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": 'iPad Air, $599, In Stock (28 units), 10.9" display, 64GB',
        "lg ultrawide 34": 'LG UltraWide 34" Monitor, $499, Out of Stock, Expected: Next week',
    }

    product_lower = product_name.lower().strip()

    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"


# Create the Product Catalog Agent
# This agent specializes in providing product information from the vendor's catalog
product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Be professional and helpful.
    """,
    tools=[get_product_info],  # Register the product lookup tool
)

print("✅ Product Catalog Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_product_info()")
print("   Ready to be exposed via A2A...")

✅ Product Catalog Agent created successfully!
   Model: gemini-2.5-flash-lite
   Tool: get_product_info()
   Ready to be exposed via A2A...


---
# get_inventory_schedules

In [6]:
# Define a inventory schedules lookup tool
# In a real system, this would query the vendor's product database
def get_inventory_schedules(product_name: str) -> str:
    """Checks stock levels and restocking schedules

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Product stock level and restocking schedule as a string
    """
    # Mock inventory database - in production, this would be a live API/DB query
    # This data is specifically for supply chain and stock logistics.
    inventory_schedules = {
        "iphone 15 pro": "Status: Low Stock. Current Level: 8 units. Next restock shipment (50 units) expected in 3 business days.",
        "samsung galaxy s24": "Status: In Stock. Current Level: 31 units. Regular weekly restocking schedule every Friday.",
        "dell xps 15": "Status: In Stock. Current Level: 45 units. High volume. Next pallet (100 units) arriving in 2 days.",
        "macbook pro 14": "Status: In Stock. Current Level: 22 units. Stable supply. Monitored automatically, no shortage expected.",
        "sony wh-1000xm5": "Status: In Stock. Current Level: 67 units. Healthy stock. Next scheduled delivery: 2 weeks.",
        "ipad air": "Status: In Stock. Current Level: 28 units. Stable. Restocks bi-weekly.",
        "lg ultrawide 34": "Status: Out of Stock. Current Level: 0 units. Supplier delay. Expected back in stock next week (approx. 7-10 days).",
        "logitech mx master 3s": "Status: Discontinued. Current Level: 0 units. This item will not be restocked. Superseded by MX Master 4.",
    }

    product_lower = product_name.lower().strip()

    if product_lower in inventory_schedules:
        return f"InventoryData for{product_name}:: {inventory_schedules[product_lower]}"
    else:
        available = ", ".join([p.title() for p in inventory_schedules.keys()])
        return f"Sorry, I don't have inventory schedule information for {product_name}. I can report on products: {available}"


# Create the Inventory Agent
# This agent specializes in providing product information from the vendor's catalog
inventory_schedules_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="inventory_schedules_agent",
    description="External vendor's inventory schedules agent that Checks stock levels and restocking schedules . ",
    instruction="""
    You are a inventory scheduler specialist from an external vendor.
    Your sole purpose is to provide stock levels and restocking schedules .
    When asked about products,you MUST use the get_inventory_schedules tool to fetch data .
    Provide clear, accurate inventory schedules data you receive from the tool.
    If asked about multiple products, look up each one.
    Be professional, concise, and helpful. Do not provide product descriptions, only inventory data..
    """,
    tools=[get_inventory_schedules],  # Register the inventory lookup tool
)

print("✅ inventory schedules Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_inventory_schedules()")
print("   Ready to be exposed via A2A...")

✅ inventory schedules Agent created successfully!
   Model: gemini-2.5-flash-lite
   Tool: get_inventory_schedules()
   Ready to be exposed via A2A...


---
# get_Shipping_details

In [7]:
# Define a Shipping details lookup tool
# In a real system, this would query the vendor's product database
def get_Shipping_details(product_name: str) -> str:
    """provides delivery estimates and tracking

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
       provides delivery estimates and tracking
    """
    # Mock shipping options database - in production, this would query a live logistics system
    # This data is product-specific (e.g., based on size, weight, warehouse location)
    shipping_details = {
        "iphone 15 pro": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $15 extra).",
        "samsung galaxy s24": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $15 extra).",
        "dell xps 15": "Oversize item. Ships from specialty warehouse. Options: Standard Ground (5-7 business days). Express shipping not available.",
        "macbook pro 14": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $20 extra).",
        "sony wh-1000xm5": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $10 extra).",
        "ipad air": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $10 extra).",
        "lg ultrawide 34": "Heavy/Oversize item. Ships from regional depot. Options: Standard Freight (7-10 business days). Note: Item is currently Out of Stock; estimates apply once restocked."
    }
        
    product_lower = product_name.lower().strip()

    if product_lower in Shipping_details:
        return f"Shipping Options for {product_name} :: {Shipping_details[product_lower]}"
    else:
        available = ", ".join([p.title() for p in Shipping_details.keys()])
        return f"Sorry, I don't have shipping information for {product_name}. I can provide estimates for products: {available}"


# Create the Shipping Agent
# This agent specializes in providing product information from the vendor's catalog
shipping_details_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="shipping_details_agent",
    description="Provides product-level shipping options and delivery estimates.",
    instruction="""
    You are a Shipping details specialist from an external vendor.
    Your purpose is to provide shipping options and delivery estimates for specific products.
    You MUST use the get_shipping_details tool to fetch this data.
    
    IMPORTANT: If a user asks for 'tracking' or 'tracking number', you MUST state that tracking numbers are generated *after* a purchase is complete and can be retrieved from an 'Order Status' agent, not this one. You only provide pre-purchase shipping estimates.
    
    Provide only the information from the tool. Be professional, concise, and helpful.
    """,
    tools=[get_Shipping_details],  # Register the product lookup tool
)

print("✅ Shipping Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_Shipping_details()")
print("   Ready to be exposed via A2A...")

✅ Shipping Agent created successfully!
   Model: gemini-2.5-flash-lite
   Tool: get_Shipping_details()
   Ready to be exposed via A2A...


# product_catalog_agent

In [8]:
# Convert the product catalog agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
product_catalog_a2a_app = to_a2a(
    product_catalog_agent, port=8001  # Port where this agent will be served
)

print("✅ Product Catalog Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8001")
print("   Agent card will be at: http://localhost:8001/.well-known/agent-card.json")
print("   Ready to start the server...")

✅ Product Catalog Agent is now A2A-compatible!
   Agent will be served at: http://localhost:8001
   Agent card will be at: http://localhost:8001/.well-known/agent-card.json
   Ready to start the server...


# Inventory_agent

In [9]:
# Convert the Invetory agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
inventory_a2a_app = to_a2a(
    inventory_schedules_agent, port=8002  # Port where this agent will be served
)

print("✅ Inventory Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8002")
print("   Agent card will be at: http://localhost:8002/.well-known/agent-card.json")
print("   Ready to start the server...")

✅ Inventory Agent is now A2A-compatible!
   Agent will be served at: http://localhost:8002
   Agent card will be at: http://localhost:8002/.well-known/agent-card.json
   Ready to start the server...


# Shipping agent

In [10]:
# Convert the product catalog agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
shipping_a2a_app = to_a2a(
    shipping_details_agent, port=8003  # Port where this agent will be served
)

print("✅ Shipping Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8003")
print("   Agent card will be at: http://localhost:8003/.well-known/agent-card.json")
print("   Ready to start the server...")

✅ Shipping Agent is now A2A-compatible!
   Agent will be served at: http://localhost:8003
   Agent card will be at: http://localhost:8003/.well-known/agent-card.json
   Ready to start the server...


# Product Catalog Agent uvicorn

In [11]:
# First, let's save the product catalog agent to a file that uvicorn can import
product_catalog_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def get_product_info(product_name: str) -> str:
    """Get product information for a given product."""
    product_catalog = {
        "iphone 15 pro": "iPhone 15 Pro, $999, Low Stock (8 units), 128GB, Titanium finish",
        "samsung galaxy s24": "Samsung Galaxy S24, $799, In Stock (31 units), 256GB, Phantom Black",
        "dell xps 15": "Dell XPS 15, $1,299, In Stock (45 units), 15.6\\" display, 16GB RAM, 512GB SSD",
        "macbook pro 14": "MacBook Pro 14\\", $1,999, In Stock (22 units), M3 Pro chip, 18GB RAM, 512GB SSD",
        "sony wh-1000xm5": "Sony WH-1000XM5 Headphones, $399, In Stock (67 units), Noise-canceling, 30hr battery",
        "ipad air": "iPad Air, $599, In Stock (28 units), 10.9\\" display, 64GB",
        "lg ultrawide 34": "LG UltraWide 34\\" Monitor, $499, Out of Stock, Expected: Next week",
    }
    
    product_lower = product_name.lower().strip()
    
    if product_lower in product_catalog:
        return f"Product: {product_catalog[product_lower]}"
    else:
        available = ", ".join([p.title() for p in product_catalog.keys()])
        return f"Sorry, I don't have information for {product_name}. Available products: {available}"

product_catalog_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="product_catalog_agent",
    description="External vendor's product catalog agent that provides product information and availability.",
    instruction="""
    You are a product catalog specialist from an external vendor.
    When asked about products, use the get_product_info tool to fetch data from the catalog.
    Provide clear, accurate product information including price, availability, and specs.
    If asked about multiple products, look up each one.
    Be professional and helpful.
    """,
    tools=[get_product_info]
)

# Create the A2A app
app = to_a2a(product_catalog_agent, port=8001)
'''

# Write the product catalog agent to a temporary file
with open("/tmp/product_catalog_server.py", "w") as f:
    f.write(product_catalog_agent_code)

print("📝 Product Catalog agent code saved to /tmp/product_catalog_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "product_catalog_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting Product Catalog Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Product Catalog Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["product_catalog_server_process"] = server_process

📝 Product Catalog agent code saved to /tmp/product_catalog_server.py
🚀 Starting Product Catalog Agent server...
   Waiting for server to be ready...
......
✅ Product Catalog Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


# inventory agent uvicorn

In [12]:
# First, let's save the inventory agent to a file that uvicorn can import
inventory_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

def get_inventory_schedules(product_name: str) -> str:
    """Checks stock levels and restocking schedules

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Product stock level and restocking schedule as a string
    """
    
    # Mock inventory database
    inventory_schedules = {
        "iphone 15 pro": "Status: Low Stock. Current Level: 8 units. Next restock shipment (50 units) expected in 3 business days.",
        "samsung galaxy s24": "Status: In Stock. Current Level: 31 units. Regular weekly restocking schedule every Friday.",
        "dell xps 15": "Status: In Stock. Current Level: 45 units. High volume. Next pallet (100 units) arriving in 2 days.",
        "macbook pro 14": "Status: In Stock. Current Level: 22 units. Stable supply. Monitored automatically, no shortage expected.",
        "sony wh-1000xm5": "Status: In Stock. Current Level: 67 units. Healthy stock. Next scheduled delivery: 2 weeks.",
        "ipad air": "Status: In Stock. Current Level: 28 units. Stable. Restocks bi-weekly.",
        "lg ultrawide 34": "Status: Out of Stock. Current Level: 0 units. Supplier delay. Expected back in stock next week (approx. 7-10 days).",
    }

    product_lower = product_name.lower().strip()

    if product_lower in inventory_schedules:
        return f"InventoryData for {product_name}: {inventory_schedules[product_lower]}"
    else:
        available = ", ".join([p.title() for p in inventory_schedules.keys()])
        return f"Sorry, I don't have inventory schedule information for '{product_name}'. I can report on: {available}"

# --- Agent Definition ---

inventory_schedules_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="inventory_schedules_agent",
    description="External vendor's inventory schedules agent that Checks stock levels and restocking schedules.",
    instruction="""
    You are an inventory schedule specialist from an external vendor.
    Your sole purpose is to provide stock levels and restocking schedules.
    When asked about products, you MUST use the get_inventory_schedules tool to fetch data.
    Provide the clear, accurate inventory schedule data you receive from the tool.
    If asked about multiple products, look up each one individually.
    Be professional, concise, and helpful. Do not provide product descriptions, only inventory data.
    """,
    tools=[get_inventory_schedules],
)

# Create the A2A app
app = to_a2a(inventory_schedules_agent, port=8002)
'''

# Write the product catalog agent to a temporary file
with open("/tmp/inventory_server.py", "w") as f:
    f.write(inventory_agent_code)

print("📝 inventory agent code saved to /tmp/inventory_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "inventory_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8002",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting inventory Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8002/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Inventory Agent server is running!")
            print(f"   Server URL: http://localhost:8002")
            print(f"   Agent card: http://localhost:8002/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["inventory_server_process"] = server_process

📝 inventory agent code saved to /tmp/inventory_server.py
🚀 Starting inventory Agent server...
   Waiting for server to be ready...
......
✅ Inventory Agent server is running!
   Server URL: http://localhost:8002
   Agent card: http://localhost:8002/.well-known/agent-card.json


# shipping agent uvicorn

In [13]:
# First, let's save the product catalog agent to a file that uvicorn can import
shipping_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


# --- Tool Definition ---

def get_shipping_details(product_name: str) -> str:
    """Provides delivery estimates and shipping options.

    Args:
        product_name: Name of the product (e.g., "iPhone 15 Pro", "MacBook Pro")

    Returns:
        Available shipping options and delivery estimates as a string
    """
    
    # Mock shipping options database
    shipping_details = {
        "iphone 15 pro": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $15 extra).",
        "samsung galaxy s24": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $15 extra).",
        "dell xps 15": "Oversize item. Ships from specialty warehouse. Options: Standard Ground (5-7 business days). Express shipping not available.",
        "macbook pro 14": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $20 extra).",
        "sony wh-1000xm5": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $10 extra).",
        "ipad air": "Ships from central warehouse. Options: Standard (3-5 business days), Express (1-2 business days, $10 extra).",
        "lg ultrawide 34": "Heavy/Oversize item. Ships from regional depot. Options: Standard Freight (7-10 business days). Note: Item is currently Out of Stock; estimates apply once restocked."
    }
        
    product_lower = product_name.lower().strip()

    if product_lower in shipping_details:
        return f"Shipping Options for {product_name}: {shipping_details[product_lower]}"
    else:
        available = ", ".join([p.title() for p in shipping_details.keys()])
        if available:
            return f"Sorry, I don't have shipping information for {product_name}. I can provide estimates for: {available}"
        else:
            return f"Sorry, I don't have shipping information for {product_name}. The shipping catalog is currently unavailable."

# --- Agent Definition ---

shipping_details_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="shipping_details_agent",
    description="Provides product-level shipping options and delivery estimates.",
    instruction="""
    You are a Shipping Details specialist from an external vendor.
    Your purpose is to provide shipping options and delivery estimates for specific products.
    You MUST use the get_shipping_details tool to fetch this data.
    
    IMPORTANT: If a user asks for 'tracking' or 'tracking number', you MUST state that tracking numbers are generated *after* a purchase is complete and can be retrieved from an 'Order Status' agent, not this one. You only provide pre-purchase shipping estimates.
    
    Provide only the information from the tool. Be professional, concise, and helpful.
    """,
    tools=[get_shipping_details],
)

# Create the A2A app
app = to_a2a(shipping_details_agent, port=8003)
'''

# Write the product catalog agent to a temporary file
with open("/tmp/shipping_server.py", "w") as f:
    f.write(shipping_agent_code)

print("📝 shipping agent code saved to /tmp/shipping_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "shipping_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8003",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting shipping Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8003/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ shipping Agent server is running!")
            print(f"   Server URL: http://localhost:8003")
            print(f"   Agent card: http://localhost:8003/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["shipping_server_process"] = server_process

📝 shipping agent code saved to /tmp/shipping_server.py
🚀 Starting shipping Agent server...
   Waiting for server to be ready...
.....
✅ shipping Agent server is running!
   Server URL: http://localhost:8003
   Agent card: http://localhost:8003/.well-known/agent-card.json


# Product catalog agent card

In [14]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Product Catalog Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Product Catalog Agent server is running (previous cell)")

📋 Product Catalog Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor's product catalog agent that provides product information and availability.",
  "name": "product_catalog_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor's product catalog agent that provides product information and availability. \n    I am a product catalog specialist from an external vendor.\n    When asked about products, use the get_product_info tool to fetch data from the catalog.\n    Provide clear, accurate product information including price, availability, and specs.\n    If asked about multiple products, look up each one.\n    Be professional and helpful.\n    ",
      "id": "product_catalog_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Get product information

# Inventory agent card

In [15]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8002/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 inventory Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Invetory Agent server is running (previous cell)")

📋 inventory Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "External vendor's inventory schedules agent that Checks stock levels and restocking schedules.",
  "name": "inventory_schedules_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "External vendor's inventory schedules agent that Checks stock levels and restocking schedules. \n    I am an inventory schedule specialist from an external vendor.\n    my sole purpose is to provide stock levels and restocking schedules.\n    When asked about products, I MUST use the get_inventory_schedules tool to fetch data.\n    Provide the clear, accurate inventory schedule data I receive from the tool.\n    If asked about multiple products, look up each one individually.\n    Be professional, concise, and helpful. Do not provide product descriptions, only inventory data.\n    ",
     

# shipping agent card

In [16]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8003/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 shipping Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Shipping Agent server is running (previous cell)")

📋 shipping Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "Provides product-level shipping options and delivery estimates.",
  "name": "shipping_details_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Provides product-level shipping options and delivery estimates. \n    I am a Shipping Details specialist from an external vendor.\n    my purpose is to provide shipping options and delivery estimates for specific products.\n    I MUST use the get_shipping_details tool to fetch this data.\n    \n    IMPORTANT: If a user asks for 'tracking' or 'tracking number', I MUST state that tracking numbers are generated *after* a purchase is complete and can be retrieved from an 'Order Status' agent, not this one. I only provide pre-purchase shipping estimates.\n    \n    Provide only the information from the tool. Be professional, con

# RemoteA2aAgent Product Catalog Agent

In [17]:
# Create a RemoteA2aAgent that connects to our Product Catalog Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_product_catalog_agent = RemoteA2aAgent(
    name="product_catalog_agent",
    description="Remote product catalog agent from external vendor that provides product information.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Product Catalog Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

✅ Remote Product Catalog Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   The Customer Support Agent can now use this like a local sub-agent!


# RemoteA2aAgent Inventory Agent

In [18]:
# Create a RemoteA2aAgent that connects to our Inventory Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_inventory_schedules_agent = RemoteA2aAgent(
    name="inventory_schedules_agent",
    description="Remote inventory agent. Use this to check stock levels and restocking schedules.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8002{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Inventory Agent proxy created!")
print(f"   Connected to: http://localhost:8002")
print(f"   Agent card: http://localhost:8002{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

✅ Remote Inventory Agent proxy created!
   Connected to: http://localhost:8002
   Agent card: http://localhost:8002/.well-known/agent-card.json
   The Customer Support Agent can now use this like a local sub-agent!


# RemoteA2aAgent Shipping Agent

In [19]:
# Create a RemoteA2aAgent that connects to our Shipping Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_shipping_details_agent = RemoteA2aAgent(
    name="shipping_details_agent",
    description="Remote shipping agent. Use this to get delivery estimates and shipping options.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8003{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote Shipping Agent proxy created!")
print(f"   Connected to: http://localhost:8003")
print(f"   Agent card: http://localhost:8003{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

✅ Remote Shipping Agent proxy created!
   Connected to: http://localhost:8003
   Agent card: http://localhost:8003/.well-known/agent-card.json
   The Customer Support Agent can now use this like a local sub-agent!


# Main Agent

In [20]:
# Now create the Customer Support Agent that uses the remote Product Catalog Agent
customer_support_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="customer_support_agent",
    description="A customer support assistant that helps customers with product inquiries and information.",
    instruction="""
    You are a friendly and professional customer support agent.
    
    When customers ask about products:
    1. Use the 'product_catalog_agent' tool to look up product information (specs, price).
    2. Use the 'inventory_schedules_agent' tool to check stock levels and restock dates.
    3. Use the 'shipping_details_agent' tool to get delivery estimates.
    4. Synthesize all this information into a clear answer about pricing, availability, specs, and shipping.
    5. If a product is out of stock, mention the expected availability from the inventory agent.
    6. Be helpful and professional!
    
    Always get information from your tools before answering customer questions.
    """,
    sub_agents=[remote_product_catalog_agent,remote_inventory_schedules_agent ,remote_shipping_details_agent ],  # Add the remote agent as a sub-agent!
)

print("✅ Customer Support Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-agents: 1 (remote Product Catalog Agent via A2A)")
print("   Sub-agents: 2 (remote Invetnory Agent via A2A)")
print("   Sub-agents: 3 (remote Shipping Agent via A2A)")
print("   Ready to help customers!")

✅ Customer Support Agent created!
   Model: gemini-2.5-flash-lite
   Sub-agents: 1 (remote Product Catalog Agent via A2A)
   Sub-agents: 2 (remote Invetnory Agent via A2A)
   Sub-agents: 3 (remote Shipping Agent via A2A)
   Ready to help customers!


# Test A2A Communication

In [21]:
async def test_a2a_communication(user_query: str):
    """
    Test the A2A communication between Customer Support Agent and Product Catalog Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the Customer Support Agent
    3. Support Agent communicates with Product Catalog Agent via A2A
    4. Displays the response

    Args:
        user_query: The question to ask the Customer Support Agent
    """
    # Setup session management (required by ADK)
    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "support_app"
    user_id = "demo_user"
    # Use unique session ID for each test to avoid conflicts
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    # CRITICAL: Create session BEFORE running agent (synchronous, not async!)
    # This pattern matches the deployment notebook exactly
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the Customer Support Agent
    # The runner manages the agent execution and session state
    runner = Runner(
        agent=customer_support_agent, app_name=app_name, session_service=session_service
    )

    # Create the user message
    # This follows the same pattern as the deployment notebook
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 Support Agent response:")
    print("-" * 60)

    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)

# Test Cases 

In [22]:
# Run the test
print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about the iPhone 15 Pro? Is it in stock?")

🧪 Testing A2A Communication...


👤 Customer: Can you tell me about the iPhone 15 Pro? Is it in stock?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.agents.remote_a2a_agent:Successfully resolved remote A2A agent: product_catalog_agent


The iPhone 15 Pro is available for $999. We currently have low stock, with only 8 units remaining. It comes with a 128GB storage capacity and a titanium finish.
------------------------------------------------------------


In [23]:
await test_a2a_communication("How long will it take to ship the Dell XPS 15?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: How long will it take to ship the Dell XPS 15?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.agents.remote_a2a_agent:Successfully resolved remote A2A agent: shipping_details_agent
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


The Dell XPS 15 has the following shipping options:

*   Standard Ground: 5-7 business days.

Please note that express shipping is not available for this item as it ships from a specialty warehouse.
------------------------------------------------------------


In [24]:
await test_a2a_communication("I want to buy the LG UltraWide 34 monitor. When will it be available and what are the shipping options?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: I want to buy the LG UltraWide 34 monitor. When will it be available and what are the shipping options?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


I'm sorry, but I could not find information for the "LG UltraWide 34 monitor." However, I did find that "Lg Ultrawide 34" is available. Would you like me to provide information on that product instead?
------------------------------------------------------------


In [25]:
await test_a2a_communication("What are the specs for the MacBook Pro 14, how many are in stock, and what are the express shipping options?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: What are the specs for the MacBook Pro 14, how many are in stock, and what are the express shipping options?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


The MacBook Pro 14" is available for $1,999. There are 22 units currently in stock. It features the M3 Pro chip with 18GB of RAM and a 512GB SSD.

Regarding express shipping, that information isn't available through my current tools. I recommend contacting our customer support team directly for details on shipping options.
------------------------------------------------------------


In [26]:
await test_a2a_communication("Can you track my order for the Samsung Galaxy S24?")

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: Can you track my order for the Samsung Galaxy S24?

🎧 Support Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.


I can't help you track your order. However, I can provide you with shipping estimates for products. If you'd like to know the estimated delivery time for a Samsung Galaxy S24, please let me know.
------------------------------------------------------------
